In [ ]:
import json
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import cv2

json_file_path = './video_info.json'

train_videos = []
train_labels = []

with open(json_file_path, 'r') as f:
    data = json.load(f)
    for item in data:
        train_videos.append(item['path'])
        train_labels.append(1 if item['status'] == 'FAKE' else 0)

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, num_classes=2)

train_video_paths, val_video_paths, train_labels_split, val_labels_split = train_test_split(
    train_videos, train_labels, test_size=0.2, random_state=42
)

def create_cnn():
    base_model = MobileNetV2(weights='imagenet', include_top=False)
    model = models.Sequential([
        TimeDistributed(base_model, input_shape=(None, 224, 224, 3)),
        TimeDistributed(GlobalAveragePooling2D())
    ])
    return model

def create_model():
    cnn_model = create_cnn()
    model = models.Sequential([
        cnn_model,
        LSTM(512, return_sequences=False, dropout=0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def load_image_frames(folder_path, sequence_length=20, input_size=(224, 224)):
    frame_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.jpg')], key=lambda x: int(os.path.splitext(x)[0]))
    frames = []
    
    for frame_file in frame_files[:sequence_length]:
        frame_path = os.path.join(folder_path, frame_file)
        frame = cv2.imread(frame_path)
        frame = cv2.resize(frame, input_size)
        frames.append(frame)

    # Pad with blank frames if fewer frames than sequence_length
    while len(frames) < sequence_length:
        frames.append(np.zeros((input_size[0], input_size[1], 3), dtype=np.uint8))
    
    return np.array(frames)

# Data generator function
def create_dataset(video_paths, labels, batch_size=8, sequence_length=20):
    def generator():
        for folder_path, label in zip(video_paths, labels):
            frames = load_image_frames(folder_path, sequence_length)
            yield frames, label  # label is now one-hot encoded with shape (2,)

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(sequence_length, 224, 224, 3), dtype=tf.uint8),
            tf.TensorSpec(shape=(2,), dtype=tf.float32)  # Label is one-hot encoded
        )
    )
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

batch_size = 4
sequence_length = 30
train_dataset = create_dataset(train_video_paths, train_labels_split, batch_size, sequence_length)
val_dataset = create_dataset(val_video_paths, val_labels_split, batch_size, sequence_length)

# Initialize and train the model
model = create_model()
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
model.summary()
model.fit(train_dataset, epochs=100, validation_data=val_dataset, callbacks=[early_stopping])

model.save("model.h5")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_36416\2627264163.py:32: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False)
C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, None, 1280)     │     2,257,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 512)            │     3,672,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,995,970 (22.87 MB)

 Trainable params: 5,961,858 (22.74 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [ ]:
import cv2
import numpy as np

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2

# Load the model
model = load_model("/content/model.h5", custom_objects={
    "TimeDistributed": TimeDistributed,
    "LSTM": LSTM,
    "GlobalAveragePooling2D": GlobalAveragePooling2D
})


def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))  # Resize as per model
        frames.append(frame)
    cap.release()
    frames = np.array(frames) / 255.0  # Normalize pixel values
    return frames

# video_frames = preprocess_video('/content/test_video.mp4')
# video_frames = video_frames.reshape(1, -1, 224, 224, 3)  # Add batch dimension
# predictions = model.predict(video_frames)
# print(predictions)